In [1]:
import torch
import torchaudio
from torchaudio import transforms
import os
import time
from fdn import DiffFDN
import pandas as pd
import numpy as np
import audio_utility as util
import json
from matplotlib import pyplot as plt
import pyfar as pf
from IPython.display import Audio

### das blöde ist hier das man eigentlich immer kernel neu starten muss nach jedem run. 
### vlt doch lieber als normale python datei

path setup und so

In [2]:
exp = "outputs/20260123-103928" ### path to exp directory
args_path = os.path.join(exp, "args.json")
### read args from json file
with open(args_path, "r") as f:
    args = json.load(f)

model_sr = args["samplerate"]
ir_length = args["ir_length"]
device = 'cpu'

# init neural net
ckpt_path = os.path.join(exp, "checkpoints/model_e16.pt") # in der datei sind die trainingsgewichte gespeichert
filepath = 'Params/'
N = args["N"]
delay_set = args["delay_set"]
filename = 'param' + '_N' + str(N) + '_d' + str(delay_set)

df = pd.read_csv(filepath+filename+'.csv', delimiter=';', nrows=N*N, dtype={'A':np.float32,'m':'Int32'})
delay_lens = torch.from_numpy(df['m'][:N].to_numpy())

net = DiffFDN(delay_lens, model_sr, ir_length) # nn init
weights = torch.load(ckpt_path, map_location=device) # load weights
net.load_state_dict(weights) # load weights ins nn
net = net.eval()


/Users/oscar/miniconda3/envs/dla-fdn/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [3]:
#hier wird die IR gefüttert und eine prediction gemacht:

### TODO loop über mehrere IRs, dann kann man epochen untereinander vergleichen
eval_filepath = "/Users/oscar/documents/Uni/Audiokommunikation/3. Semester/DLA/Impulse Responses/eval/SC_MC_EIG_3.wav"
eval_ir, sr = torchaudio.load(eval_filepath)

if sr != model_sr:
    tf = transforms.Resample(sr, model_sr)
    eval_ir = tf(eval_ir)
    sr = model_sr

if eval_ir.shape[0] != 1:
    eval_ir = eval_ir.mean(dim=0, keepdim=True)

eval_ir = util.pad_crop(eval_ir, sr, ir_length)
if eval_ir.ndim == 2:        # [C, T] zu [B, C, T]
    eval_ir = eval_ir.unsqueeze(0)

z = util.get_frequency_samples(int(args['ir_length']*args['samplerate']))

t = time.time()
pred = net(eval_ir, z)
pred = pred.squeeze(0)
print(f"ir_length: {ir_length} seconds")
print(f"generated pred in {np.round(np.abs(t-time.time()), 2)} seconds")

print(f"pred.shape: {pred.shape}")
print(f"eval_ir.shape: {eval_ir.shape}")


ir_length: 2.0 seconds
generated pred in 0.54 seconds
pred.shape: torch.Size([1, 96000])
eval_ir.shape: torch.Size([1, 1, 96000])


In [4]:
# plot
pred_np = util.normalize(pred)
pred_np = pred_np.squeeze(0).detach().cpu().numpy()
eval_np = eval_ir.squeeze(0).detach().cpu().numpy()
print("eval:")
display(Audio(eval_np, rate=48000))
print("pred")
display(Audio(pred_np, rate=48000))
print(f"pred.shape: {pred.shape}")
print(f"eval_ir.shape: {eval_ir.shape}")

times = np.zeros(len(pred_np))
eval_sig = pf.Signal([eval_np.flatten(),times.flatten()],sampling_rate=model_sr)
pred_sig = pf.Signal([pred_np.flatten(),times.flatten()],sampling_rate=model_sr)

pf.plot.time_freq(eval_sig, label="eval", alpha=0.3)
pf.plot.time_freq(pred_sig, label="pred", alpha=1.)
plt.legend()
plt.show()

eval:


pred


/Users/oscar/miniconda3/envs/dla-fdn/lib/python3.11/site-packages/IPython/lib/display.py:173: ComplexWarning: Casting complex values to real discards the imaginary part
  data = np.array(data, dtype=float)


pred.shape: torch.Size([1, 96000])
eval_ir.shape: torch.Size([1, 1, 96000])


ValueError: time data is complex, set is_complex flag or pass real-valued data.